In [4]:
import pandas as pd

messages1=pd.read_csv("/content/Womens Clothing E-Commerce Reviews.csv")

In [5]:
messages1

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33.0,NaN,Absolutely wonderful - silky and sexy and comf...,4.0,1.0,0.0,Initmates,Intimate,Intimates
1,1,1080,34.0,NaN,Love this dress! it's sooo pretty. i happene...,5.0,1.0,4.0,General,Dresses,Dresses
2,2,1077,60.0,Some major design flaws,I had such high hopes for this dress and reall...,3.0,0.0,0.0,General,Dresses,Dresses
3,3,1049,50.0,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5.0,1.0,0.0,General Petite,Bottoms,Pants
4,4,847,47.0,Flattering shirt,This shirt is very flattering to all due to th...,5.0,1.0,6.0,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
20328,20328,1078,26.0,Beautiful fit!,"As a teacher, it's so great to be able to find...",5.0,1.0,23.0,General,Dresses,Dresses
20329,20329,872,46.0,Comfy top,I bought the black and love it. i bought both ...,5.0,1.0,0.0,General,Tops,Knits
20330,20330,836,46.0,Easy breezy summer top,I absolutely love this blouse! it's easily one...,5.0,1.0,4.0,General,Tops,Blouses
20331,20331,1049,38.0,NaN,I would not recommend these pants if you have ...,2.0,0.0,3.0,General,Bottoms,Pants


In [6]:
messages1.shape

(20333, 11)

In [7]:
messages=messages1.rename(columns={"Review Text": "message","Recommended IND": "label"})

In [8]:
messages=messages[["message","label"]]


In [9]:
messages.shape

(20333, 2)

In [10]:
label_counts = messages['label'].value_counts()
# zero_label_counts = messages['label'].value_counts()
print(label_counts)

1.0    16705
0.0     3627
Name: label, dtype: int64


In [11]:
# i will only work with 5000 samples
messages = messages.head(10000)

In [12]:
messages=messages.rename(columns={"Review Text": "message","Recommended IND": "label"})

In [13]:
messages.shape

(10000, 2)

In [14]:
label_count=messages.value_counts()
label_count #1: 325, 0: 27

message                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               label
"vegan suede" makes these sound fancier than they are. i love them and they're super cozy and a nice change from plain leggings, but they're just a microfiber legging which makes the price about 20 too high even for retailer. still... i'm sure i'll wear them plenty. i usually wear a 2-4 and the xs fit me, which makes me think they'd be too large for a size 0/xs.                                                                                                                                

In [15]:
messages

,message,label
0,Absolutely wonderful - silky and sexy and comf...,1.0
1,Love this dress! it's sooo pretty. i happene...,1.0
2,I had such high hopes for this dress and reall...,0.0
3,"I love, love, love this jumpsuit. it's fun, fl...",1.0
4,This shirt is very flattering to all due to th...,1.0
...,...,...
9995,"This dress is quite adorable - the pattern, sl...",1.0
9996,Love this dress! bodice and lace at bottom giv...,1.0
9997,The cut is perfect and the rose color is so pr...,1.0
9998,I love this shirt so i bought both colors. i w...,1.0


In [16]:
messages=messages[["message","label"]]


In [17]:
messages

,message,label
0,Absolutely wonderful - silky and sexy and comf...,1.0
1,Love this dress! it's sooo pretty. i happene...,1.0
2,I had such high hopes for this dress and reall...,0.0
3,"I love, love, love this jumpsuit. it's fun, fl...",1.0
4,This shirt is very flattering to all due to th...,1.0
...,...,...
9995,"This dress is quite adorable - the pattern, sl...",1.0
9996,Love this dress! bodice and lace at bottom giv...,1.0
9997,The cut is perfect and the rose color is so pr...,1.0
9998,I love this shirt so i bought both colors. i w...,1.0


In [18]:
messages['message'].loc[451]

'This vest is very warm and soft. it is actually a fleece that almost appears as wool. the plaid is beautiful-a pretty soft pink with the black and charcoal colors. i ordered a size large and the fit is okay. an xl would have been too big under the arms and in the bust area however, i would have liked it a little looser in the bottom half. i kept the large as there is no closure to the vest. it just hangs open.'

In [19]:
messages.isnull().sum()

message    352
label        0
dtype: int64

In [20]:
#Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [22]:
for i in range(0, len(messages)):
    if not isinstance(messages['message'][i], str):
        print(type(messages['message'][i]))


<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class '

hence thatswhy we were facing issue in preprocessing out most of the inputs were float but all should be string

# Preprocessing
Stop words removal,
lowerization,
tokenization

In [23]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Assuming you have the necessary imports for re, stopwords, and PorterStemmer

ps = PorterStemmer()

corpus = []
for i in range(0, len(messages)):
    # Convert non-string values to strings
    if not isinstance(messages['message'][i], str):
        messages['message'][i] = str(messages['message'][i])

# preprocessing
    # Remove non-alphabetic characters, convert to lowercase, and perform stemming
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


<ipython-input-23-fffa32b80c6c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messages['message'][i] = str(messages['message'][i])


In [24]:
corpus

['absolut wonder silki sexi comfort',
 'love dress sooo pretti happen find store glad bc never would order onlin bc petit bought petit love length hit littl knee would definit true midi someon truli petit',
 'high hope dress realli want work initi order petit small usual size found outrag small small fact could zip reorder petit medium ok overal top half comfort fit nice bottom half tight layer sever somewhat cheap net layer imo major design flaw net layer sewn directli zipper c',
 'love love love jumpsuit fun flirti fabul everi time wear get noth great compliment',
 'shirt flatter due adjust front tie perfect length wear leg sleeveless pair well cardigan love shirt',
 'love traci rees dress one petit feet tall usual wear p brand dress pretti packag lot dress skirt long full overwhelm small frame stranger alter shorten narrow skirt would take away embellish garment love color idea style work return dress',
 'ade basket hte last mintu see would look like person store pick went teh darkl

# BOW

In [25]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [26]:
y=pd.get_dummies(messages['label'])
y=y.iloc[:,1].values

In [28]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

#  Instantiate the RandomOverSampler
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)  # You can adjust the sampling strategy
X_resampled, y_resampled = oversampler.fit_resample(X, y)


In [29]:
label_counts_new = messages['label'].value_counts()
# zero_label_counts = messages['label'].value_counts()
print(label_counts_new)

1.0    8242
0.0    1758
Name: label, dtype: int64


In [ ]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [73]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [74]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [75]:
from sklearn.metrics import accuracy_score,classification_report

In [76]:
score=accuracy_score(y_test,y_pred)
print(score)

0.8835


In [77]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.69      0.65      0.67       366
           1       0.92      0.94      0.93      1634

    accuracy                           0.88      2000
   macro avg       0.81      0.79      0.80      2000
weighted avg       0.88      0.88      0.88      2000



# Tf-IDF

In [78]:
# Creating the TFIDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=2500)
X = tv.fit_transform(corpus).toarray()

In [79]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [80]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [81]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [82]:
score=accuracy_score(y_test,y_pred)
print(score)

0.8465


In [83]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.12      0.89      0.21        46
           1       1.00      0.85      0.91      1954

    accuracy                           0.85      2000
   macro avg       0.56      0.87      0.56      2000
weighted avg       0.98      0.85      0.90      2000



## Word2vec Implementation

In [84]:
!pip install gensim

In [85]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [86]:


vec_king = wv['king']

In [87]:
vec_king

array([ 1.25976562e-01,  2.97851562e-02,  8.60595703e-03,  1.39648438e-01,
       -2.56347656e-02, -3.61328125e-02,  1.11816406e-01, -1.98242188e-01,
        5.12695312e-02,  3.63281250e-01, -2.42187500e-01, -3.02734375e-01,
       -1.77734375e-01, -2.49023438e-02, -1.67968750e-01, -1.69921875e-01,
        3.46679688e-02,  5.21850586e-03,  4.63867188e-02,  1.28906250e-01,
        1.36718750e-01,  1.12792969e-01,  5.95703125e-02,  1.36718750e-01,
        1.01074219e-01, -1.76757812e-01, -2.51953125e-01,  5.98144531e-02,
        3.41796875e-01, -3.11279297e-02,  1.04492188e-01,  6.17675781e-02,
        1.24511719e-01,  4.00390625e-01, -3.22265625e-01,  8.39843750e-02,
        3.90625000e-02,  5.85937500e-03,  7.03125000e-02,  1.72851562e-01,
        1.38671875e-01, -2.31445312e-01,  2.83203125e-01,  1.42578125e-01,
        3.41796875e-01, -2.39257812e-02, -1.09863281e-01,  3.32031250e-02,
       -5.46875000e-02,  1.53198242e-02, -1.62109375e-01,  1.58203125e-01,
       -2.59765625e-01,  

In [97]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [98]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [99]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()

    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [100]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [101]:
corpus

['absolutely wonderful silky sexy comfortable',
 'love dress sooo pretty happened find store glad bc never would ordered online bc petite bought petite love length hit little knee would definitely true midi someone truly petite',
 'high hope dress really wanted work initially ordered petite small usual size found outrageously small small fact could zip reordered petite medium ok overall top half comfortable fit nicely bottom half tight layer several somewhat cheap net layer imo major design flaw net layer sewn directly zipper c',
 'love love love jumpsuit fun flirty fabulous every time wear get nothing great compliment',
 'shirt flattering due adjustable front tie perfect length wear legging sleeveless pair well cardigan love shirt',
 'love tracy reese dress one petite foot tall usually wear p brand dress pretty package lot dress skirt long full overwhelmed small frame stranger alteration shortening narrowing skirt would take away embellishment garment love color idea style work return

In [103]:
import nltk
nltk.download('punkt')

words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [104]:
words

[['absolutely', 'wonderful', 'silky', 'sexy', 'comfortable'],
 ['love',
  'dress',
  'sooo',
  'pretty',
  'happened',
  'find',
  'store',
  'glad',
  'bc',
  'never',
  'would',
  'ordered',
  'online',
  'bc',
  'petite',
  'bought',
  'petite',
  'love',
  'length',
  'hit',
  'little',
  'knee',
  'would',
  'definitely',
  'true',
  'midi',
  'someone',
  'truly',
  'petite'],
 ['high',
  'hope',
  'dress',
  'really',
  'wanted',
  'work',
  'initially',
  'ordered',
  'petite',
  'small',
  'usual',
  'size',
  'found',
  'outrageously',
  'small',
  'small',
  'fact',
  'could',
  'zip',
  'reordered',
  'petite',
  'medium',
  'ok',
  'overall',
  'top',
  'half',
  'comfortable',
  'fit',
  'nicely',
  'bottom',
  'half',
  'tight',
  'layer',
  'several',
  'somewhat',
  'cheap',
  'net',
  'layer',
  'imo',
  'major',
  'design',
  'flaw',
  'net',
  'layer',
  'sewn',
  'directly',
  'zipper'],
 ['love',
  'love',
  'love',
  'jumpsuit',
  'fun',
  'flirty',
  'fabulous',

In [126]:
len(words)

10000

In [105]:
import gensim

In [106]:
### Lets train Word2vec from scratch
model=gensim.models.Word2Vec(words,window=5,min_count=2)

In [107]:
model.wv.index_to_key

['dress',
 'fit',
 'size',
 'love',
 'top',
 'like',
 'color',
 'look',
 'wear',
 'great',
 'would',
 'fabric',
 'small',
 'really',
 'ordered',
 'perfect',
 'one',
 'little',
 'flattering',
 'shirt',
 'soft',
 'well',
 'back',
 'cute',
 'nice',
 'comfortable',
 'bought',
 'beautiful',
 'bit',
 'large',
 'material',
 'sweater',
 'work',
 'length',
 'much',
 'also',
 'jean',
 'run',
 'got',
 'long',
 'short',
 'petite',
 'medium',
 'waist',
 'quality',
 'retailer',
 'skirt',
 'pretty',
 'think',
 'even',
 'store',
 'feel',
 'lb',
 'usually',
 'make',
 'tried',
 'sleeve',
 'way',
 'pant',
 'good',
 'go',
 'made',
 'could',
 'get',
 'cut',
 'big',
 'see',
 'black',
 'style',
 'super',
 'still',
 'summer',
 'right',
 'true',
 'design',
 'white',
 'arm',
 'wearing',
 'fall',
 'purchased',
 'model',
 'however',
 'online',
 'perfectly',
 'tight',
 'enough',
 'time',
 'jacket',
 'sale',
 'looked',
 'shoulder',
 'bottom',
 'front',
 'definitely',
 'blue',
 'though',
 'loved',
 'price',
 'review

In [108]:
model.corpus_count

10000

In [109]:
model.epochs

5

In [114]:
model.wv.similar_by_word('awesome')

[('effortless', 0.9891857504844666),
 ('fresh', 0.9891389608383179),
 ('lbd', 0.9885360598564148),
 ('poncho', 0.988347053527832),
 ('notch', 0.988186776638031),
 ('boho', 0.9875732660293579),
 ('flannel', 0.9873949289321899),
 ('comfort', 0.9872851371765137),
 ('formal', 0.9862727522850037),
 ('sleek', 0.9862315058708191)]

In [125]:
# here you can note that when you will check similarity of a word with different word you will only get results for those words which are present in our vacab otherwise you
# will get error hence this is limitation of word2vec

In [115]:
model.wv['kid'].shape

(100,)

# AvgWord2Vec
It basically will combine vectors of all the words into a single capture which will represent the context of entire doc hence it becomes easy to work on simple data and also becomes comutationaly effective

In [116]:
import numpy as np
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)

    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)




In [117]:
!pip install tqdm

In [118]:
from tqdm import tqdm

In [119]:
words[73]

['love',
 'top',
 'loose',
 'comfortable',
 'sheer',
 'need',
 'cami',
 'run',
 'little',
 'large',
 'size',
 'want',
 'tailored',
 'fit',
 'criticism',
 'catch',
 'sharp',
 'jewelry',
 'belt',
 'pull',
 'careful']

In [120]:
type(model.wv.index_to_key)

list

words is our whole 10000 word vocabulary here basically I am applying avgword2vec technique on entire vocab

In [ ]:
#apply for the entire sentences
X=[]
for i in tqdm(range(len(words))):
    print("Hello",i)
    X.append(avg_word2vec(words[i]))



In [127]:
type(X)

list

In [122]:
X_new=np.array(X) # converting  resulted list of avgword2vec into array

In [128]:
X_new.shape

(10000, 100)

In [129]:
X_new[1] # it is representing 100 dim vector representation of my first sentence of corpus

array([-2.73199379e-01,  5.55540800e-01, -1.56061118e-02, -1.26572892e-01,
        3.85282069e-01, -6.64354205e-01,  3.77285004e-01,  7.38218784e-01,
       -6.58042490e-01, -2.58709878e-01, -9.78966877e-02, -5.49150884e-01,
        3.15565467e-02,  2.18903255e-02,  3.80080521e-01, -8.07293534e-01,
        8.36455524e-02, -1.24471886e-02,  1.22448839e-02, -6.58812702e-01,
        8.02456662e-02,  2.45469660e-01,  2.06186593e-01, -4.06080961e-01,
       -2.13711075e-02, -8.83131996e-02,  7.00565502e-02, -4.53295350e-01,
       -2.53959298e-01, -9.69241336e-02,  3.41838628e-01, -8.07184726e-02,
        2.20497623e-02, -4.84827697e-01, -2.02179521e-01,  1.28293812e-01,
       -4.52629626e-02, -7.45428503e-02, -1.47160634e-01, -7.65272260e-01,
        1.94095045e-01, -6.80646822e-02, -1.04374282e-01, -2.37962231e-02,
        6.68991357e-02,  1.93350688e-02, -2.05553859e-01,  1.03466056e-01,
        3.98542315e-01,  6.63628802e-02,  3.04475248e-01, -3.60151201e-01,
       -1.28323473e-02, -

In [172]:
# sent=corpus_small=corpus[1:100]
sent=corpus

In [173]:
print(len(sent))

10000


In [174]:
!pip install --upgrade pip


In [139]:
pip install --upgrade setuptools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [141]:
pip uninstall tensorflow tensorflow-gpu


Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.15.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
y
  Successfully uninstalled tensorflow-2.15.0


In [142]:
pip install tensorflow-gpu


  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [146]:
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 4.0 MB/s eta 0:00:00


In [147]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [148]:
##tensorflow >2.0
from tensorflow.keras.preprocessing.text import one_hot

In [149]:
### Vocabulary size
voc_size=1000

# One hot representation

In [175]:
onehot_repr=[one_hot(words,voc_size)for words in sent]
print(onehot_repr)

[[686, 976, 809, 240, 766], [479, 959, 490, 614, 38, 855, 158, 902, 466, 462, 827, 875, 486, 466, 939, 452, 939, 479, 846, 582, 985, 766, 827, 359, 637, 15, 505, 777, 939], [426, 112, 959, 609, 386, 867, 250, 875, 939, 890, 576, 564, 846, 415, 890, 890, 127, 932, 372, 717, 939, 614, 538, 374, 71, 246, 766, 210, 464, 455, 246, 354, 5, 645, 877, 660, 369, 5, 121, 374, 360, 66, 369, 5, 513, 198, 551, 255], [479, 479, 479, 786, 191, 798, 595, 721, 478, 123, 828, 193, 405, 863], [266, 750, 12, 33, 62, 484, 356, 846, 123, 60, 262, 667, 632, 282, 479, 266], [479, 386, 995, 959, 273, 939, 35, 770, 707, 123, 485, 451, 959, 614, 654, 724, 959, 984, 3, 933, 289, 890, 644, 95, 471, 110, 125, 984, 827, 321, 636, 721, 11, 479, 267, 82, 160, 867, 936, 959], [661, 952, 948, 660, 280, 114, 827, 488, 379, 744, 158, 37, 495, 572, 321, 267, 722, 948, 267, 609, 202, 850, 113, 286, 900, 255, 985, 137, 948, 788, 948, 347, 564, 110, 939, 520, 803, 31, 11, 945, 818, 348, 551, 984, 433, 900, 187, 41], [875, 150

In [176]:
max_size = max(len(sublist) for sublist in onehot_repr)

print("Length of the longest list:", max_size)

Length of the longest list: 58


Hence it is clear that max length of sentences is 50 so i should fix 50 as maxlen parameter

# Word Embedding representation

In [151]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [152]:
import numpy as np

In [177]:
## pre padding
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[  0   0   0 ... 809 240 766]
 [  0   0   0 ... 505 777 939]
 [  0   0 426 ... 198 551 255]
 ...
 [  0   0   0 ... 695 915 267]
 [  0   0   0 ... 210 405 229]
 [  0   0   0 ... 632 712 356]]


In [178]:
len(embedded_docs)

10000

In [179]:
## 20 feature dimesnions
dim=40


In [180]:
model=Sequential()
model.add(Embedding(voc_size,20,input_length=sent_length))
model.compile('adam','mse')

In [181]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 20)            20000     
                                                                 
Total params: 20000 (78.12 KB)
Trainable params: 20000 (78.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [182]:
corpus[0]

'absolutely wonderful silky sexy comfortable'

In [183]:
embedded_docs[0] # vector representation of first sentence with max length 50 and along with pre padding

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0, 686, 976, 809, 240, 766], dtype=int32)

In [184]:
model.predict(embedded_docs[0]) # in output we can also see each sentence is representing by 20 features(note 20 values )

2/2 [==============================] - 0s 4ms/step


array([[ 0.01570214, -0.01110853,  0.01205913, -0.02118833,  0.02145784,
        -0.02915077,  0.03959406, -0.04332773,  0.03361892,  0.03649731,
         0.01299173, -0.00905155,  0.04624236, -0.00997566,  0.00774472,
         0.00671411, -0.0138701 , -0.01132774,  0.00769929,  0.02159831],
       [ 0.01570214, -0.01110853,  0.01205913, -0.02118833,  0.02145784,
        -0.02915077,  0.03959406, -0.04332773,  0.03361892,  0.03649731,
         0.01299173, -0.00905155,  0.04624236, -0.00997566,  0.00774472,
         0.00671411, -0.0138701 , -0.01132774,  0.00769929,  0.02159831],
       [ 0.01570214, -0.01110853,  0.01205913, -0.02118833,  0.02145784,
        -0.02915077,  0.03959406, -0.04332773,  0.03361892,  0.03649731,
         0.01299173, -0.00905155,  0.04624236, -0.00997566,  0.00774472,
         0.00671411, -0.0138701 , -0.01132774,  0.00769929,  0.02159831],
       [ 0.01570214, -0.01110853,  0.01205913, -0.02118833,  0.02145784,
        -0.02915077,  0.03959406, -0.04332773,  

In [185]:
print(model.predict(embedded_docs))

313/313 [==============================] - 0s 1ms/step
[[[ 0.01570214 -0.01110853  0.01205913 ... -0.01132774  0.00769929
    0.02159831]
  [ 0.01570214 -0.01110853  0.01205913 ... -0.01132774  0.00769929
    0.02159831]
  [ 0.01570214 -0.01110853  0.01205913 ... -0.01132774  0.00769929
    0.02159831]
  ...
  [ 0.03628987 -0.04725805 -0.0421734  ...  0.04155276  0.00837332
    0.02584526]
  [ 0.02887881  0.03238286 -0.03587534 ... -0.03460979  0.00015094
    0.03533534]
  [ 0.04385981  0.01299557  0.04643459 ...  0.02807431  0.04271686
    0.0133507 ]]

 [[ 0.01570214 -0.01110853  0.01205913 ... -0.01132774  0.00769929
    0.02159831]
  [ 0.01570214 -0.01110853  0.01205913 ... -0.01132774  0.00769929
    0.02159831]
  [ 0.01570214 -0.01110853  0.01205913 ... -0.01132774  0.00769929
    0.02159831]
  ...
  [-0.01428575 -0.04413865 -0.0475302  ...  0.00263876 -0.02616416
   -0.03315441]
  [ 0.03504806  0.010604   -0.01885624 ... -0.01660377 -0.00920769
    0.04908717]
  [-0.00333115 -0.

In [186]:
## Creating model
embedding_vector_features=40 ##features representation
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))


In [187]:
from tensorflow.keras.layers import LSTM, Dense
model2.add(LSTM(100))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model2.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 50, 40)            40000     
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 96501 (376.96 KB)
Trainable params: 96501 (376.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [188]:
len(embedded_docs),y.shape

(10000, (10000,))

In [189]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [190]:
X_final.shape,y_final.shape

((10000, 50), (10000,))

In [191]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [193]:
### Finally Training
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
105/105 [==============================] - 11s 53ms/step - loss: 0.4398 - accuracy: 0.8224 - val_loss: 0.3267 - val_accuracy: 0.8615
Epoch 2/10
105/105 [==============================] - 6s 53ms/step - loss: 0.2787 - accuracy: 0.8819 - val_loss: 0.3122 - val_accuracy: 0.8676
Epoch 3/10
105/105 [==============================] - 7s 71ms/step - loss: 0.2458 - accuracy: 0.8975 - val_loss: 0.3095 - val_accuracy: 0.8682
Epoch 4/10
105/105 [==============================] - 9s 83ms/step - loss: 0.2302 - accuracy: 0.9072 - val_loss: 0.3262 - val_accuracy: 0.8648
Epoch 5/10
105/105 [==============================] - 8s 79ms/step - loss: 0.2079 - accuracy: 0.9163 - val_loss: 0.3408 - val_accuracy: 0.8685
Epoch 6/10
105/105 [==============================] - 5s 49ms/step - loss: 0.1917 - accuracy: 0.9249 - val_loss: 0.3558 - val_accuracy: 0.8606
Epoch 7/10
105/105 [==============================] - 7s 66ms/step - loss: 0.1769 - accuracy: 0.9331 - val_loss: 0.3604 - val_accuracy: 0.859

# from LSTM also accuracy is 86% on training data

# Adding Dropout Layer

In [194]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [195]:
y_pred=model.predict(X_test)

104/104 [==============================] - 2s 13ms/step


In [202]:
y_pred=np.where(y_pred > 0.4, 1,0) ##AUC ROC Curve

In [203]:
from sklearn.metrics import confusion_matrix

In [204]:
confusion_matrix(y_test,y_pred)

array([[ 576,    0],
       [2724,    0]])

In [205]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.17454545454545456

very bad accuracy and the reason might be we are using only 10,000 data samples

In [206]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.17      1.00      0.30       576
           1       0.00      0.00      0.00      2724

    accuracy                           0.17      3300
   macro avg       0.09      0.50      0.15      3300
weighted avg       0.03      0.17      0.05      3300



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
